# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [1]:
def read_col_file(filename):
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [46]:
import itertools

In [96]:
k = 1 #Ширина окрестности

def basic_local_search(graph):
    def edges_between(v1, v2):
        res = 0
        for v in v1:
            for u in v2:
                if M[v][u]:
                    res += 1
        return res
    
    V = list(graph[0])
    n = len(V)
    E = list(graph[1])
    M = [[False] * (n + 1) for i in range(n + 1)] # Для удобства запишем ребра в матрицу смежности
    for e in E:
        M[e[0]][e[1]] = True
        M[e[1]][e[0]] = True
    V1 = set(V[:len(V) // 2])
    V2 = set(V[len(V) // 2:])
    while True:
        max_delta = 0
        best_v1 = []
        best_v2 = []
        for v_s in itertools.product(V1, repeat=k):
            for u_s in itertools.product(V2, repeat=k):
                v1_s = set(v_s)
                v2_s = set(u_s)
                if len(v1_s) != len(v2_s):
                    continue
                V1_P = V1 - v1_s
                V2_P = V2 - v2_s
                delta = edges_between(V1_P, v2_s) + edges_between(v1_s, V2_P) - \
                      edges_between(v1_s, V1_P) - edges_between(v2_s, V2_P)
                if delta > max_delta:
                    max_delta = delta
                    best_v1 = v1_s
                    best_v2 = v2_s
        if max_delta == 0:
            break
        new_V1 = (V1 - best_v1) | best_v2
        new_V2 = (V2 - best_v2) | best_v1
        V1 = new_V1
        V2 = new_V2
    print('V1: ', V1)
    print('edges between: ', edges_between(V1, V2))
    return V1

In [91]:
graph = read_col_file('myciel3.col')
basic_local_search(graph)

V1:  {3, 4, 5, 6, 10}
edges between:  8


{3, 4, 5, 6, 10}

In [92]:
graph = read_col_file('myciel4.col')
basic_local_search(graph)

V1:  {3, 4, 5, 7, 8, 9, 10, 12, 14, 16, 22}
edges between:  27


{3, 4, 5, 7, 8, 9, 10, 12, 14, 16, 22}

In [100]:
graph = read_col_file('myciel5.col')
print(basic_local_search(graph))

V1:  {6, 8, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 29, 31, 34, 45, 46}
edges between:  92
{6, 8, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 29, 31, 34, 45, 46}


In [98]:
graph = read_col_file('myciel6.col')
print(basic_local_search(graph))

V1:  {2, 4, 7, 9, 12, 13, 16, 17, 18, 19, 20, 21, 24, 26, 27, 29, 30, 31, 32, 33, 35, 37, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 58, 59, 63, 69, 70, 71, 73, 75, 81, 82, 93, 94}
edges between:  292
{2, 4, 7, 9, 12, 13, 16, 17, 18, 19, 20, 21, 24, 26, 27, 29, 30, 31, 32, 33, 35, 37, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 58, 59, 63, 69, 70, 71, 73, 75, 81, 82, 93, 94}


In [99]:
graph = read_col_file('myciel7.col')
print(basic_local_search(graph))

V1:  {3, 5, 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 39, 40, 41, 42, 43, 44, 46, 49, 50, 51, 52, 54, 56, 57, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 77, 78, 79, 80, 84, 85, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99, 100, 105, 106, 107, 109, 110, 111, 116, 117, 118, 119, 121, 122, 123, 129, 132, 141, 142, 144, 145, 146, 147, 153, 165, 188, 189, 190}
edges between:  913
{3, 5, 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 39, 40, 41, 42, 43, 44, 46, 49, 50, 51, 52, 54, 56, 57, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 72, 73, 74, 75, 77, 78, 79, 80, 84, 85, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99, 100, 105, 106, 107, 109, 110, 111, 116, 117, 118, 119, 121, 122, 123, 129, 132, 141, 142, 144, 145, 146, 147, 153, 165, 188, 189, 190}
